In [1]:
!git clone https://github.com/Youbadawy/FinRL-Library.git

Cloning into 'FinRL-Library'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 2391 (delta 8), reused 15 (delta 4), pack-reused 2366
Receiving objects: 100% (2391/2391), 29.10 MiB | 26.23 MiB/s, done.
Resolving deltas: 100% (1402/1402), done.


In [2]:
%cd FinRL-Library/
!pip install -r requirements.txt
!pip install nest-asyncio

/content/FinRL-Library
     |████████████████████████████████| 92kB 3.8MB/s 
     |████████████████████████████████| 153kB 8.3MB/s 
     |████████████████████████████████| 51kB 4.9MB/s 
     |████████████████████████████████| 1.4MB 8.2MB/s 
     |████████████████████████████████| 2.0MB 24.9MB/s 
     |████████████████████████████████| 5.5MB 38.1MB/s 
     |████████████████████████████████| 61kB 4.7MB/s 
     |████████████████████████████████| 368kB 37.1MB/s 
     |████████████████████████████████| 256kB 46.8MB/s 
     |████████████████████████████████| 3.2MB 42.5MB/s 
     |████████████████████████████████| 1.3MB 41.5MB/s 
     |████████████████████████████████| 143kB 33.2MB/s 
     |████████████████████████████████| 81kB 7.0MB/s 
     |████████████████████████████████| 235kB 43.1MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.55-py2.py3-none-any.whl size=22616 sha256=d7f9985c14c04048d4fe55a036bf98146b0c6d40b9c6ea49c40411b595f634f1
  Stored in directory: /root/.cache/pip/whee

In [3]:
!pip install -U ipython
# !pip install colorama

     |████████████████████████████████| 788kB 4.2MB/s 
ERROR: jupyter-console 5.2.0 has requirement prompt-toolkit<2.0.0,>=1.0.0, but you'll have prompt-toolkit 3.0.16 which is incompatible.
ERROR: google-colab 1.0.0 has requirement ipython~=5.5.0, but you'll have ipython 7.16.1 which is incompatible.
  Found existing installation: ipython 5.5.0
    Uninstalling ipython-5.5.0:
      Successfully uninstalled ipython-5.5.0


In [1]:
%cd FinRL-Library/

/content/FinRL-Library


In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime

%matplotlib inline
from finrl.config import config
from finrl.marketdata.yahoodownloader import YahooDownloader
from finrl.preprocessing.preprocessors import FeatureEngineer
from finrl.preprocessing.data import data_split
from finrl.env.env_stocktrading import StockTradingEnv
from finrl.model.models import DRLAgent
from finrl.trade.backtest import backtest_stats, backtest_plot, get_daily_return, get_baseline

from pprint import pprint

import sys
sys.path.append("../FinRL-Library")

import itertools

/usr/local/lib/python3.6/dist-packages/pyfolio/pos.py:28: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  ' to position notionals.'


In [4]:
import os
if not os.path.exists("./" + config.DATA_SAVE_DIR):
    os.makedirs("./" + config.DATA_SAVE_DIR)
if not os.path.exists("./" + config.TRAINED_MODEL_DIR):
    os.makedirs("./" + config.TRAINED_MODEL_DIR)
if not os.path.exists("./" + config.TENSORBOARD_LOG_DIR):
    os.makedirs("./" + config.TENSORBOARD_LOG_DIR)
if not os.path.exists("./" + config.RESULTS_DIR):
    os.makedirs("./" + config.RESULTS_DIR)

In [5]:
from finrl.config.configuration import Configuration
from finrl.config.directory_operations import create_userdata_dir
from finrl.commands import start_download_cryptodata, start_download_stockdata

In [15]:
#### CREATE USER DATA DIRECTORY IN DESIGNATED PATH, IF NO NAME INDICATED DEFAULT TO user_data
####### create dir to false if only to check existence of directory
create_userdata_dir("./user_data",create_dir=True)


# ###### Pull Configuration File (using finrl/config/configuration.py)
config = Configuration.from_files(["config.json"])

In [16]:
from finrl.commands import start_download_cryptodata, start_download_stockdata, start_list_markets

#ARGS_LIST_PAIRS = ["exchange", "print_list", "list_pairs_print_json", "print_one_column",
#                   "print_csv", "base_currencies", "quote_currencies", "list_pairs_all"]

ARGS_LIST_PAIRS = {"exchange":config["exchange"]["name"], "quote_currencies":"BNB"}

x = start_list_markets(ARGS_LIST_PAIRS)


Exchange Binance has 111 active markets with B, N, B as quote currencies:
+-----------+------------+--------+---------+----------+-----------+
|        Id |     Symbol |   Base |   Quote |   Active |   Is pair |
|-----------+------------+--------+---------+----------+-----------|
|   AAVEBNB |   AAVE/BNB |   AAVE |     BNB |     True |      True |
|    ADABNB |    ADA/BNB |    ADA |     BNB |     True |      True |
|   ALGOBNB |   ALGO/BNB |   ALGO |     BNB |     True |      True |
|  ALPHABNB |  ALPHA/BNB |  ALPHA |     BNB |     True |      True |
|   ANKRBNB |   ANKR/BNB |   ANKR |     BNB |     True |      True |
|    ANTBNB |    ANT/BNB |    ANT |     BNB |     True |      True |
|   ARPABNB |   ARPA/BNB |   ARPA |     BNB |     True |      True |
|   ATOMBNB |   ATOM/BNB |   ATOM |     BNB |     True |      True |
|    AVABNB |    AVA/BNB |    AVA |     BNB |     True |      True |
|   AVAXBNB |   AVAX/BNB |   AVAX |     BNB |     True |      True |
|    AXSBNB |    AXS/BNB |  

In [75]:
from finrl.tools.coin_search import *
import json
#Search top Selling Coins based on Volume
coins = coinSearch("BNB", top=20)
print(coins)
#Add them to config file Pair_whitelist
coins_to_json("./config.json", coins)

# reintialize config
config = Configuration.from_files(["config.json"])

#make sure the pairs are equal...
print(config["exchange"]["pair_whitelist"])

['HOT/BNB', 'WIN/BNB', 'MBL/BNB', 'IOST/BNB', 'BTT/BNB', 'DGB/BNB', 'RVN/BNB', 'JST/BNB', 'COS/BNB', 'IQ/BNB', 'MITH/BNB', 'ONE/BNB', 'ANKR/BNB', 'CHZ/BNB', 'CELR/BNB', 'XRP/BNB', 'XLM/BNB', 'OGN/BNB', 'MFT/BNB', 'FET/BNB']
['HOT/BNB', 'WIN/BNB', 'MBL/BNB', 'IOST/BNB', 'BTT/BNB', 'DGB/BNB', 'RVN/BNB', 'JST/BNB', 'COS/BNB', 'IQ/BNB', 'MITH/BNB', 'ONE/BNB', 'ANKR/BNB', 'CHZ/BNB', 'CELR/BNB', 'XRP/BNB', 'XLM/BNB', 'OGN/BNB', 'MFT/BNB', 'FET/BNB']


In [76]:
ARGS_DOWNLOAD_DATA = {'config': ['config.json'], 'datadir': None, 
                      'user_data_dir': None, 'pairs': None, 'pairs_file': None, 
                      'days': 365, 'timerange': None, 
                      'download_trades': False, 'exchange': 'binance', 
                      'timeframes': ['4h'], 'erase': False, 
                      'dataformat_ohlcv': None, 'dataformat_trades': None}

# ######## downloads data to our local data repository as dictated by our config, or we could overide it using 'datadir'
start_download_cryptodata(ARGS_DOWNLOAD_DATA)

In [77]:
from finrl.data.fetchdata import FetchData
import pandas as pd
from finrl.config import TimeRange

In [78]:
df = FetchData(config).fetch_data_crypto()
print(df.head)

coin HOT_BNB completed...
coin WIN_BNB completed...
coin MBL_BNB completed...
coin IOST_BNB completed...
coin BTT_BNB completed...
coin DGB_BNB completed...
coin RVN_BNB completed...
coin JST_BNB completed...
coin COS_BNB completed...
coin IQ_BNB completed...
coin MITH_BNB completed...
coin ONE_BNB completed...
coin ANKR_BNB completed...
coin CHZ_BNB completed...
coin CELR_BNB completed...
coin XRP_BNB completed...
coin XLM_BNB completed...
coin OGN_BNB completed...
coin MFT_BNB completed...
coin FET_BNB completed...
(41151, 7)
<bound method NDFrame.head of                     date      open      high  ...     close      volume      tic
0    2020-02-13 00:00:00  0.000032  0.000034  ...  0.000034  15549743.0  HOT_BNB
1    2020-02-13 04:00:00  0.000034  0.000035  ...  0.000033  18295454.0  HOT_BNB
2    2020-02-13 08:00:00  0.000034  0.000036  ...  0.000036  30499857.0  HOT_BNB
3    2020-02-13 12:00:00  0.000035  0.000036  ...  0.000034  32951545.0  HOT_BNB
4    2020-02-13 16:00:00  0.000

In [79]:
df.head()

,date,open,high,low,close,volume,tic
0,2020-02-13 00:00:00,0.000032,0.000034,0.000031,0.000034,15549743.0,HOT_BNB
1,2020-02-13 04:00:00,0.000034,0.000035,0.000033,0.000033,18295454.0,HOT_BNB
2,2020-02-13 08:00:00,0.000034,0.000036,0.000033,0.000036,30499857.0,HOT_BNB
3,2020-02-13 12:00:00,0.000035,0.000036,0.000033,0.000034,32951545.0,HOT_BNB
4,2020-02-13 16:00:00,0.000034,0.000034,0.000033,0.000034,3948011.0,HOT_BNB


In [80]:
df.sort_values(['date','tic'],ignore_index=True).head()

fe = FeatureEngineer(
                    use_technical_indicator=True,
                    tech_indicator_list = config["TECHNICAL_INDICATORS_LIST"],
                    use_turbulence=False,
                    user_defined_feature = False)

processed = fe.preprocess_data(df)
processed.columns

Successfully added technical indicators


Index(['date', 'open', 'high', 'low', 'close', 'volume', 'tic', 'macd',
       'boll_ub', 'boll_lb', 'rsi_30', 'cci_30', 'dx_30', 'close_30_sma',
       'close_60_sma'],
      dtype='object')

In [81]:
print(processed.head())
processed.shape

                 date      open  ...  close_30_sma  close_60_sma
0 2020-02-13 00:00:00  0.000032  ...      0.000034      0.000034
1 2020-02-13 04:00:00  0.000034  ...      0.000034      0.000034
2 2020-02-13 08:00:00  0.000034  ...      0.000034      0.000034
3 2020-02-13 12:00:00  0.000035  ...      0.000034      0.000034
4 2020-02-13 16:00:00  0.000034  ...      0.000034      0.000034

[5 rows x 15 columns]


(41151, 15)

In [82]:
print(processed.tail())
processed.shape

                    date      open  ...  close_30_sma  close_60_sma
2186 2021-02-11 12:00:00  0.002038  ...      0.000011      0.000013
2187 2021-02-11 16:00:00  0.001903  ...      0.000011      0.000013
2188 2021-02-11 20:00:00  0.001925  ...      0.000011      0.000013
2189 2021-02-12 00:00:00  0.001887  ...      0.000011      0.000012
2190 2021-02-12 04:00:00  0.001833  ...      0.000011      0.000012

[5 rows x 15 columns]


(41151, 15)

In [88]:
list_ticker = processed["tic"].unique().tolist()
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max(), freq="4h").astype(str))
combination = list(itertools.product(list_date,list_ticker))
list_date

['2020-02-13 00:00:00',
 '2020-02-13 04:00:00',
 '2020-02-13 08:00:00',
 '2020-02-13 12:00:00',
 '2020-02-13 16:00:00',
 '2020-02-13 20:00:00',
 '2020-02-14 00:00:00',
 '2020-02-14 04:00:00',
 '2020-02-14 08:00:00',
 '2020-02-14 12:00:00',
 '2020-02-14 16:00:00',
 '2020-02-14 20:00:00',
 '2020-02-15 00:00:00',
 '2020-02-15 04:00:00',
 '2020-02-15 08:00:00',
 '2020-02-15 12:00:00',
 '2020-02-15 16:00:00',
 '2020-02-15 20:00:00',
 '2020-02-16 00:00:00',
 '2020-02-16 04:00:00',
 '2020-02-16 08:00:00',
 '2020-02-16 12:00:00',
 '2020-02-16 16:00:00',
 '2020-02-16 20:00:00',
 '2020-02-17 00:00:00',
 '2020-02-17 04:00:00',
 '2020-02-17 08:00:00',
 '2020-02-17 12:00:00',
 '2020-02-17 16:00:00',
 '2020-02-17 20:00:00',
 '2020-02-18 00:00:00',
 '2020-02-18 04:00:00',
 '2020-02-18 08:00:00',
 '2020-02-18 12:00:00',
 '2020-02-18 16:00:00',
 '2020-02-18 20:00:00',
 '2020-02-19 00:00:00',
 '2020-02-19 04:00:00',
 '2020-02-19 08:00:00',
 '2020-02-19 12:00:00',
 '2020-02-19 16:00:00',
 '2020-02-19 20:

In [89]:
processed["date"] = processed["date"].astype(str)
processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.fillna(0)
processed_full.sort_values(['date','tic'],ignore_index=True).head(10)

,date,tic,open,high,low,close,volume,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma
0,2020-02-13 00:00:00,ANKR_BNB,0.000073,0.000074,0.000069,0.000073,2516584.0,0.0,0.000034,0.000033,0.0,66.666667,100.0,0.000034,0.000034
1,2020-02-13 00:00:00,BTT_BNB,0.000019,0.000020,0.000019,0.000019,138086304.0,0.0,0.000034,0.000033,0.0,66.666667,100.0,0.000034,0.000034
2,2020-02-13 00:00:00,CELR_BNB,0.000179,0.000182,0.000176,0.000182,3780071.0,0.0,0.000034,0.000033,0.0,66.666667,100.0,0.000034,0.000034
3,2020-02-13 00:00:00,CHZ_BNB,0.000493,0.000510,0.000488,0.000502,408073.0,0.0,0.000034,0.000033,0.0,66.666667,100.0,0.000034,0.000034
4,2020-02-13 00:00:00,COS_BNB,0.000400,0.000407,0.000392,0.000402,202190.0,0.0,0.000034,0.000033,0.0,66.666667,100.0,0.000034,0.000034
5,2020-02-13 00:00:00,DGB_BNB,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000
6,2020-02-13 00:00:00,FET_BNB,0.002006,0.002014,0.001963,0.001998,247432.0,0.0,0.000034,0.000033,0.0,66.666667,100.0,0.000034,0.000034
7,2020-02-13 00:00:00,HOT_BNB,0.000032,0.000034,0.000031,0.000034,15549743.0,0.0,0.000034,0.000033,0.0,66.666667,100.0,0.000034,0.000034
8,2020-02-13 00:00:00,IOST_BNB,0.000301,0.000309,0.000296,0.000309,993408.0,0.0,0.000034,0.000033,0.0,66.666667,100.0,0.000034,0.000034
9,2020-02-13 00:00:00,IQ_BNB,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000


In [90]:
processed_full.head(5)

,date,tic,open,high,low,close,volume,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma
0,2020-02-13 00:00:00,HOT_BNB,0.000032,0.000034,0.000031,0.000034,15549743.0,0.0,0.000034,0.000033,0.0,66.666667,100.0,0.000034,0.000034
1,2020-02-13 00:00:00,WIN_BNB,0.000005,0.000005,0.000005,0.000005,149131156.0,0.0,0.000034,0.000033,0.0,66.666667,100.0,0.000034,0.000034
2,2020-02-13 00:00:00,MBL_BNB,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000
3,2020-02-13 00:00:00,IOST_BNB,0.000301,0.000309,0.000296,0.000309,993408.0,0.0,0.000034,0.000033,0.0,66.666667,100.0,0.000034,0.000034
4,2020-02-13 00:00:00,BTT_BNB,0.000019,0.000020,0.000019,0.000019,138086304.0,0.0,0.000034,0.000033,0.0,66.666667,100.0,0.000034,0.000034


In [91]:
processed_full.tail()

,date,tic,open,high,low,close,volume,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma
43835,2021-02-12 04:00:00,XRP_BNB,0.004520,0.004520,0.004360,0.004480,1879686.5,-5.664967e-07,0.000015,0.000007,43.969435,-39.855452,28.33043,0.000011,0.000012
43836,2021-02-12 04:00:00,XLM_BNB,0.003674,0.003769,0.003582,0.003666,378084.0,-5.664967e-07,0.000015,0.000007,43.969435,-39.855452,28.33043,0.000011,0.000012
43837,2021-02-12 04:00:00,OGN_BNB,0.003100,0.003140,0.003030,0.003140,83444.1,-5.664967e-07,0.000015,0.000007,43.969435,-39.855452,28.33043,0.000011,0.000012
43838,2021-02-12 04:00:00,MFT_BNB,0.000159,0.000180,0.000156,0.000172,50172888.0,-5.664967e-07,0.000015,0.000007,43.969435,-39.855452,28.33043,0.000011,0.000012
43839,2021-02-12 04:00:00,FET_BNB,0.001833,0.002043,0.001810,0.001980,177638.0,-5.664967e-07,0.000015,0.000007,43.969435,-39.855452,28.33043,0.000011,0.000012


In [101]:
#### DATA SPLIT TRAIN TRADE RATIO 70/30

trade_len = int(len(processed_full)*0.7)
train_len = int(len(processed_full)*0.3+1)
total = trade_len+train_len
print(f'trade length:{trade_len}, train length: {train}, for total len of {total} of {len(processed_full)}')

trade length:30673, train length: 13146.0, for total len of 43820 of 43820


In [110]:
processed_full.date.factorize()[0]

array([   0,    0,    0, ..., 2190, 2190, 2190])

In [107]:
train = data_split(processed_full, processed_full.date.min(),processed_full.date.loc[trade_len])
trade = data_split(processed_full, processed_full.date.loc[trade_len],processed_full.date.max())
print(len(train))
print(len(trade))

30640
13160


In [ ]:
# processed_imputed = processed_full[processed_full.columns[processed_full.isna().any().tolist()]]

# processed_imputed = processed_imputed.replace([np.inf, -np.inf], np.nan)

In [ ]:
# from sklearn.impute import KNNImputer

# imputer = KNNImputer(n_neighbors=1)
# imputed = imputer.fit_transform(processed_imputed)

# df_knn = pd.DataFrame(imputed, columns=processed_imputed.columns.tolist())

# for i in processed_imputed.columns.tolist():
#     processed_imputed[i] = df_knn[i]

In [ ]:
# print(processed_imputed.isna().any())

open            False
high            False
low             False
close           False
volume          False
macd            False
boll_ub         False
boll_lb         False
rsi_30          False
cci_30          False
dx_30           False
close_30_sma    False
close_60_sma    False
dtype: bool


In [ ]:
# processed_imputed.head()

,open,high,low,close,volume,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma
1,0.000744,0.000800,0.000696,0.000740,3.560429e+06,0.000000e+00,0.000005,0.000005,100.000000,66.666667,100.00000,0.000005,0.000005
9,0.085056,0.092740,0.077999,0.083463,7.923040e+07,-5.542309e-08,0.000005,0.000004,46.680829,-33.411206,25.08211,0.000004,0.000004
15,0.085056,0.092740,0.077999,0.083463,7.923040e+07,-5.542309e-08,0.000005,0.000004,46.680829,-33.411206,25.08211,0.000004,0.000004
6,0.001993,0.002173,0.001975,0.002014,1.538142e+06,0.000000e+00,0.000005,0.000005,100.000000,66.666667,100.00000,0.000005,0.000005
10,0.085056,0.092740,0.077999,0.083463,7.923040e+07,-5.542309e-08,0.000005,0.000004,46.680829,-33.411206,25.08211,0.000004,0.000004


In [ ]:
# processed_imputed.tail()

,open,high,low,close,volume,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma
7291,0.000154,0.000155,0.000102,0.000115,144796641.0,-1.027257e-07,0.000003,0.000002,40.610895,-193.348225,51.696729,0.000002,0.000002
7288,0.000204,0.000216,0.000142,0.000174,40932841.0,-1.027257e-07,0.000003,0.000002,40.610895,-193.348225,51.696729,0.000002,0.000002
7280,0.000185,0.000185,0.000136,0.000145,14528466.0,-1.027257e-07,0.000003,0.000002,40.610895,-193.348225,51.696729,0.000002,0.000002
7299,0.000979,0.000991,0.000704,0.000804,9634840.0,-1.027257e-07,0.000003,0.000002,40.610895,-193.348225,51.696729,0.000002,0.000002
7296,0.000428,0.000430,0.000329,0.000359,66127973.0,-1.027257e-07,0.000003,0.000002,40.610895,-193.348225,51.696729,0.000002,0.000002


In [ ]:
# from sklearn.model_selection import train_test_split 
# X = processed_imputed.astype(float).to_numpy()
# train, trade = train_test_split(X, test_size=0.33, shuffle=False)
# train = pd.DataFrame(train)
# trade = pd.DataFrame(trade)
# print(train.shape)
# print(trade.shape)
# # train = data_split(processed_full, '2018-05-16','2020-05-16')
# # trade = data_split(processed_full, '2020-05-17','2021-02-10')

(4891, 13)
(2409, 13)


In [ ]:
# processed_full.date.head()

1     2020-02-12
9     2020-02-12
15    2020-02-12
6     2020-02-12
10    2020-02-12
Name: date, dtype: object

In [ ]:
# train[["date","tic"]] = processed_full[["date","tic"]].iloc[:len(train),:]
# train.columns = processed_full.columns
# train = train.sort_values(["date", "tic"], ignore_index=True)
# train.index = train.date.factorize()[0]
# trade[["date","tic"]] = processed_full[["date","tic"]].iloc[:len(trade),:]
# trade.columns = processed_full.columns
# trade = trade.sort_values(["date", "tic"], ignore_index=True)
# trade.index = trade.date.factorize()[0]

In [113]:
train.head()

,date,tic,open,high,low,close,volume,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma
0,2020-02-13 00:00:00,ANKR_BNB,0.000073,0.000074,0.000069,0.000073,2516584.0,0.0,0.000034,0.000033,0.0,66.666667,100.0,0.000034,0.000034
0,2020-02-13 00:00:00,BTT_BNB,0.000019,0.000020,0.000019,0.000019,138086304.0,0.0,0.000034,0.000033,0.0,66.666667,100.0,0.000034,0.000034
0,2020-02-13 00:00:00,CELR_BNB,0.000179,0.000182,0.000176,0.000182,3780071.0,0.0,0.000034,0.000033,0.0,66.666667,100.0,0.000034,0.000034
0,2020-02-13 00:00:00,CHZ_BNB,0.000493,0.000510,0.000488,0.000502,408073.0,0.0,0.000034,0.000033,0.0,66.666667,100.0,0.000034,0.000034
0,2020-02-13 00:00:00,COS_BNB,0.000400,0.000407,0.000392,0.000402,202190.0,0.0,0.000034,0.000033,0.0,66.666667,100.0,0.000034,0.000034


In [114]:
trade.head()

,date,tic,open,high,low,close,volume,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma
0,2020-10-25 12:00:00,ANKR_BNB,0.000309,0.000311,0.000304,0.000309,2706102.0,-3.232500e-08,0.000017,0.000016,48.883772,-115.05199,6.142612,0.000017,0.000016
0,2020-10-25 12:00:00,BTT_BNB,0.000011,0.000011,0.000011,0.000011,23613679.0,-3.232500e-08,0.000017,0.000016,48.883772,-115.05199,6.142612,0.000017,0.000016
0,2020-10-25 12:00:00,CELR_BNB,0.000142,0.000142,0.000140,0.000141,31027.0,-3.232500e-08,0.000017,0.000016,48.883772,-115.05199,6.142612,0.000017,0.000016
0,2020-10-25 12:00:00,CHZ_BNB,0.000354,0.000359,0.000349,0.000356,412976.0,-3.232500e-08,0.000017,0.000016,48.883772,-115.05199,6.142612,0.000017,0.000016
0,2020-10-25 12:00:00,COS_BNB,0.000217,0.000219,0.000216,0.000219,64177.0,-3.232500e-08,0.000017,0.000016,48.883772,-115.05199,6.142612,0.000017,0.000016


In [115]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30640 entries, 0 to 1531
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   date          30640 non-null  object 
 1   tic           30640 non-null  object 
 2   open          30640 non-null  float64
 3   high          30640 non-null  float64
 4   low           30640 non-null  float64
 5   close         30640 non-null  float64
 6   volume        30640 non-null  float64
 7   macd          30640 non-null  float64
 8   boll_ub       30640 non-null  float64
 9   boll_lb       30640 non-null  float64
 10  rsi_30        30640 non-null  float64
 11  cci_30        30640 non-null  float64
 12  dx_30         30640 non-null  float64
 13  close_30_sma  30640 non-null  float64
 14  close_60_sma  30640 non-null  float64
dtypes: float64(13), object(2)
memory usage: 3.7+ MB


In [116]:
len(train.tic.unique())

20

In [117]:
config["TECHNICAL_INDICATORS_LIST"]

['macd',
 'boll_ub',
 'boll_lb',
 'rsi_30',
 'cci_30',
 'dx_30',
 'close_30_sma',
 'close_60_sma']

In [118]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(config["TECHNICAL_INDICATORS_LIST"])*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

env_kwargs = {
    "hmax": 100, 
    "initial_amount": 1000000, 
    "buy_cost_pct": 0.001,
    "sell_cost_pct": 0.001,
    "state_space": state_space, 
    "stock_dim": stock_dimension, 
    "tech_indicator_list": config["TECHNICAL_INDICATORS_LIST"], 
    "action_space": stock_dimension, 
    "reward_scaling": 1e-4
    
}
e_train_gym = StockTradingEnv(df = train, **env_kwargs)

Stock Dimension: 20, State Space: 201


In [119]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


In [120]:
agent = DRLAgent(env = env_train)
PPO_PARAMS = {
    "n_steps": 2048,
    "ent_coef": 0.01,
    "learning_rate": 0.00025,
    "batch_size": 128,
}
model_ppo = agent.get_model("ppo",model_kwargs = PPO_PARAMS)


{'n_steps': 2048, 'ent_coef': 0.01, 'learning_rate': 0.00025, 'batch_size': 128}
Using cpu device


In [121]:
trained_ppo = agent.train_model(model=model_ppo, 
                             tb_log_name='ppo',
                             total_timesteps=100000)

Logging to tensorboard_log/ppo/ppo_1
----------------------------------
| environment/        |          |
|    portfolio_value  | 1e+06    |
|    total_cost       | 3.51     |
|    total_reward     | -61.6    |
|    total_reward_pct | -0.00616 |
|    total_trades     | 27296    |
| time/               |          |
|    fps              | 149      |
|    iterations       | 1        |
|    time_elapsed     | 13       |
|    total_timesteps  | 2048     |
----------------------------------
----------------------------------------
| environment/            |            |
|    portfolio_value      | 1e+06      |
|    total_cost           | 3.45       |
|    total_reward         | -53.1      |
|    total_reward_pct     | -0.00531   |
|    total_trades         | 27366      |
| time/                   |            |
|    fps                  | 157        |
|    iterations           | 2          |
|    time_elapsed         | 26         |
|    total_timesteps      | 4096       |
| train/        

In [122]:
e_trade_gym = StockTradingEnv(df = trade, **env_kwargs)
# env_trade, obs_trade = e_trade_gym.get_sb_env()

df_account_value, df_actions = DRLAgent.DRL_prediction(
    model=model_ppo, 
    environment = e_trade_gym)

hit end!


In [123]:
df_account_value.shape

(658, 2)

In [124]:
df_account_value.head()

,date,account_value
0,2020-10-25 12:00:00,1.000000e+06
1,2020-10-25 16:00:00,1.000000e+06
2,2020-10-25 20:00:00,1.000000e+06
3,2020-10-26 00:00:00,1.000000e+06
4,2020-10-26 04:00:00,1.000000e+06


In [125]:
df_actions.head()

,ANKR_BNB,BTT_BNB,CELR_BNB,CHZ_BNB,COS_BNB,DGB_BNB,FET_BNB,HOT_BNB,IOST_BNB,IQ_BNB,JST_BNB,MBL_BNB,MFT_BNB,MITH_BNB,OGN_BNB,ONE_BNB,RVN_BNB,WIN_BNB,XLM_BNB,XRP_BNB
0,100,0,0,0,100,0,37,0,100,0,73,10,0,100,94,0,0,0,60,100
1,-60,0,26,0,-100,0,85,0,100,0,87,-10,0,83,-94,0,0,0,24,-27
2,-40,0,100,0,0,0,100,0,100,100,-70,100,57,49,39,42,0,0,27,100
3,15,100,-50,32,51,100,64,0,100,-100,90,-51,41,-100,-16,-42,0,18,-100,-49
4,0,-6,-76,-7,100,-100,60,23,-100,88,85,-49,-53,100,-6,0,16,100,45,-14


In [126]:
print("==============Get Backtest Results===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value)
perf_stats_all = pd.DataFrame(perf_stats_all)
# perf_stats_all.to_csv("./"+config.RESULTS_DIR+"/perf_stats_all_"+now+'.csv')

==============Get Backtest Results===========
Annual return         -0.000037
Cumulative returns    -0.000096
Annual volatility      0.000062
Sharpe ratio          -0.590158
Calmar ratio          -0.221633
Stability              0.501955
Max drawdown          -0.000165
Omega ratio            0.860416
Sortino ratio         -0.757088
Skew                        NaN
Kurtosis                    NaN
Tail ratio             0.956235
Daily value at risk   -0.000008
dtype: float64
